### Single AWE ANcode Decoder A=131 Synthesizer

In [1]:
import math
import numpy as np

def bits_needed(number):
    return number.bit_length()

In [2]:
GF131_positive = [1, 2, 4, 8, 16, 32, 64, 128, 125, 119, 107, 83, 35, 70, 9, 18, 36, 72, 13, 26, 52, 104, 77, 23, 46, 92, 53, 106, 81, 31, 62, 124, 117, 103, 75, 19, 38, 76, 21, 42, 84, 37, 74, 17, 34, 68, 5, 10, 20, 40, 80, 29, 58, 116, 101, 71, 11, 22, 44, 88, 45, 90, 49, 98, 65]
GF131_negative = [130, 129, 127, 123, 115, 99, 67, 3, 6, 12, 24, 48, 96, 61, 122, 113, 95, 59, 118, 105, 79, 27, 54, 108, 85, 39, 78, 25, 50, 100, 69, 7, 14, 28, 56, 112, 93, 55, 110, 89, 47, 94, 57, 114, 97, 63, 126, 121, 111, 91, 51, 102, 73, 15, 30, 60, 120, 109, 87, 43, 86, 41, 82, 33, 66]
print(np.size(GF131_positive))
print(np.size(GF131_negative))

65
65


In [3]:
plus_values131 = list(range(1, len(GF131_positive) + 1))
minus_values131 = [-i for i in range(1, len(GF131_negative) + 1)]
print(plus_values131)
print(minus_values131)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]
[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65]


### 選取特定的位元數:

In [5]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data = (1 << 52) - 1        ### 52bits
A = 131
input = data * A
bits = bits_needed(input)

print(input)
print("AN資料的位元數:", bits)
print(bits_needed(A))
print(bits_needed(data))
print(bits_needed(bits))


589971551185534845
AN資料的位元數: 60
8
52
6


In [6]:
#### SEC_AN Decoder
# 2025/5/13
#-------------------------------------
# Used to do Conventional SEC, corrected errors by AWE
# input: remainder, output: AWE
Bw = bits
Bn = bits_needed(data)
BA = bits_needed(A)
B_location = bits_needed(bits)
#-------------------------------------
fn = 'SECdecoder' +"_AWE_"+ str(Bn) + "bits" +'.v'
f = open(fn, 'w')

f.write('// Product (AN) Code Conventional SECdecoder\n')
f.write('// ' + fn + '\n')
f.write('// Used to do SEC by Galois Field, and corrected AWE\n')
f.write('// Received remainder r, output single AWE.\n')
f.write('module SECdecoder_AWE_%dbits(r, AWE);\n'% (Bn))
f.write('input \t[%d:0]\tr;\n' % (BA-1))
f.write('output\treg\tsigned\t[%d:0]\tAWE;\n' % (np.size(GF131_positive)))

f.write('always@(*) begin\n')
f.write('\tcase(r)\n')
for i in range(np.size(GF131_positive)):
    f.write("\t\t%d: AWE = +(1 << %d);\n" % (GF131_positive[i] ,plus_values131[i]-1))
for j in range(np.size(GF131_negative)):
    f.write("\t\t%d: AWE = -(1 << %d);\n" % (GF131_negative[j] ,plus_values131[j]-1))
f.write("\t\tdefault: AWE = 0;\n")
f.write('\tendcase\n')
f.write('end\n\n')

f.write('endmodule\n')
f.close();
print('Module file %s is generated.\n' % (fn))

Module file SECdecoder_AWE_52bits.v is generated.



In [8]:
#### SEC_AN Decoder
# 2025/5/13
#-------------------------------------
# Used to do Conventional SEC, corrected errors by location, not AWE
# input: remainder, output: location
Bw = bits
Bn = bits_needed(data)
BA = bits_needed(A)
B_location = bits_needed(bits)
#-------------------------------------
fn = 'SECdecoder' +"_location_"+ str(Bn) + "bits" +'.v'
f = open(fn, 'w')

f.write('// Product (AN) Code Conventional SECdecoder\n')
f.write('// ' + fn + '\n')
f.write('// Used to do SEC by Galois Field, and corrected by location, not AWE\n')
f.write('// Received remainder r, output error location.\n')
f.write('module SECdecoder_location_%dbits(r, l);\n'% (Bn))
f.write('input \t[%d:0]\tr;\n' % (BA-1))
f.write('output\treg\tsigned\t[%d:0]\tl;\n' % (B_location))

f.write('always@(*) begin\n')
f.write('\tcase(r)\n')
for i in range(np.size(GF131_positive)):
    f.write("\t\t%d: l = %+d;\n" % (GF131_positive[i] ,plus_values131[i]))
for j in range(np.size(GF131_negative)):
    f.write("\t\t%d: l = %+d;\n" % (GF131_negative[j] ,minus_values131[j]))
f.write("\t\tdefault: l = 0;\n")
f.write('\tendcase\n')
f.write('end\n\n')

f.write('endmodule\n')
f.close();
print('Module file %s is generated.\n' % (fn))

Module file SECdecoder_location_52bits.v is generated.

